In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, '/cellar/users/dlaub/projects/log_euclidean_signature/dsc291-final-project')

In [37]:
import torch
from les.les import les_dist_comp, les_desc_comp, _build_graph
from les_pytorch.les import LES
import numpy as np

In [162]:
rng = np.random.default_rng(0)
x1 = rng.normal(0, 1, (10, 1))
x2 = x1 * 2 + rng.normal(0, 0.5, (10, 1))
x1t = torch.as_tensor(x1)
x2t = torch.as_tensor(x2)
x1t.requires_grad = True
x2t.requires_grad = True

In [134]:
np.testing.assert_allclose(_build_graph(x1), LES()._heat_kernel(x1t).numpy())

In [139]:
np.random.seed(0)
f1 = les_desc_comp(x1)
f1t = LES()._log_eigenvalues(LES()._heat_kernel(x1t))
np.testing.assert_allclose(f1, f1t.numpy())

np.random.seed(0)
f2 = les_desc_comp(x2)
f2t = LES()._log_eigenvalues(LES()._heat_kernel(x2t))
np.testing.assert_allclose(f2, f2t.numpy())

les_d = les_dist_comp(f1, f2)
les_dt = ((f1t - f2t)**2).sum().sqrt()
np.testing.assert_allclose(les_d, les_dt.numpy())

In [161]:
np.random.seed(0)
f1 = les_desc_comp(x1)
f2 = les_desc_comp(x2)
les_d = les_dist_comp(f1, f2)

np.random.seed(0)
les_dt = LES()(x1t, x2t)

np.testing.assert_allclose(les_d, les_dt.detach().numpy())

In [158]:
torch.autograd.set_detect_anomaly(True)

In [172]:
np.random.seed(0)
les_dt = LES()(x1t, x2t)
les_dt.backward()